## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-24-20-50-53 +0000,nypost,The Post photographer who moonlights as DJ to ...,https://nypost.com/2025/09/24/us-news/the-post...
1,2025-09-24-20-49-08 +0000,nyt,"U.N. General Assembly Updates: China, for Firs...",https://www.nytimes.com/live/2025/09/24/world/...
2,2025-09-24-20-49-04 +0000,nyt,"At U.N. Climate Summit, U.S. Stands Alone",https://www.nytimes.com/2025/09/24/climate/uni...
3,2025-09-24-20-47-45 +0000,nyt,"Saudi Grand Mufti, an Archconservative in a Ch...",https://www.nytimes.com/2025/09/23/world/middl...
4,2025-09-24-20-46-26 +0000,nypost,Hooker from hell fed 3 Johns lethal dose of fe...,https://nypost.com/2025/09/24/us-news/hooker-f...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2322/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
168,trump,53
259,kimmel,17
93,new,15
48,ice,14
258,jimmy,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
264,2025-09-24-01-43-00 +0000,wsj,When Trump announced changes to the H-1B visa ...,https://www.wsj.com/politics/policy/air-india-...,114
139,2025-09-24-14-05-20 +0000,nypost,Trump threatens to sue ABC over Jimmy Kimmel’s...,https://nypost.com/2025/09/24/us-news/trump-th...,112
240,2025-09-24-05-04-00 +0000,wsj,Kimmel Makes Emotional Return to ABC With Trum...,https://www.wsj.com/business/media/jimmy-kimme...,107
210,2025-09-24-09-30-00 +0000,wsj,Stephen Miran set out his case for lower inter...,https://www.wsj.com/economy/central-banking/st...,97
185,2025-09-24-10-22-53 +0000,bbc,Kimmel pulls no punches as he sends defiant me...,https://www.bbc.com/news/articles/c0r09qdnd4yo...,90


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
264,114,2025-09-24-01-43-00 +0000,wsj,When Trump announced changes to the H-1B visa ...,https://www.wsj.com/politics/policy/air-india-...
134,77,2025-09-24-14-37-59 +0000,nypost,Here’s what Jimmy Kimmel said — and if he apol...,https://nypost.com/2025/09/24/media/jimmy-kimm...
18,59,2025-09-24-20-25-24 +0000,nypost,"Mom of Joshua Jahn, Dallas ICE facility shoote...",https://nypost.com/2025/09/24/us-news/joshua-j...
232,56,2025-09-24-07-04-00 +0000,wsj,China said it will no longer seek new special ...,https://www.wsj.com/economy/trade/china-to-sto...
116,49,2025-09-24-15-56-05 +0000,nypost,Scott Bessent warns that the feds won’t bail o...,https://nypost.com/2025/09/24/business/scott-b...
128,41,2025-09-24-15-06-31 +0000,bbc,Israeli forces near Gaza City centre as Hamas ...,https://www.bbc.com/news/articles/c4gk9dreldxo...
184,38,2025-09-24-10-24-08 +0000,cbc,Typhoon Ragasa makes landfall in China after k...,https://www.cbc.ca/news/world/typhoon-ragasa-c...
199,37,2025-09-24-10-00-00 +0000,nypost,Vivek Ramaswamy tells ‘Pod Force One’ US may h...,https://nypost.com/2025/09/24/us-news/vivek-ra...
281,32,2025-09-23-23-34-55 +0000,nyt,"In a Sudden Shift, Trump Says Ukraine Can Win ...",https://www.nytimes.com/2025/09/23/us/politics...
246,31,2025-09-24-04-07-38 +0000,latimes,Jimmy Kimmel returns to ABC with emotional mon...,https://www.latimes.com/entertainment-arts/tv/...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
